# Scenario 03B — Incremental Help Notebook (AWS IAM & CloudTrail)

This notebook provides **optional, step‑by‑step hints** for Scenario 03B.

Each section includes:
- A **goal**
- A **first hint** (conceptual)
- A **second hint** (more concrete)
- An optional **reveal cell** with example patterns

Use this notebook only when you need it — the main scenario notebook is designed to be completed independently.

## 1. Understanding the AWS logs

**Goal:** Understand what `cloud_iam.csv` and `cloud_api.csv` represent.

### First Hint
Think of `cloud_iam.csv` as configuration and identity changes, and `cloud_api.csv` as activity and usage.

### Second Hint
Use `.head()` and `.info()` to inspect columns like `user`, `role`, `event_name`, `resource`, `region`, and `timestamp`.

### Reveal (optional)

In [ ]:
# Example exploration
cloud_iam_df.head(), cloud_iam_df.info(), cloud_api_df.head(), cloud_api_df.info()

## 2. Timestamp normalization

    **Goal:** Ensure timestamps are parsed correctly and usable for correlation.

### First Hint
Use `pd.to_datetime()` with `utc=True` to parse ISO‑8601 timestamps.

### Second Hint
If you see a trailing `Z`, remove it before parsing. Then add `date` and `hour` columns for easier grouping.

### Reveal (optional)

In [ ]:
# Example timestamp normalization
cloud_iam_df['timestamp'] = pd.to_datetime(
    cloud_iam_df['timestamp'].astype(str).str.replace('Z', '', regex=False),
    utc=True,
    errors='coerce'
)
cloud_iam_df['hour'] = cloud_iam_df['timestamp'].dt.hour

cloud_api_df['timestamp'] = pd.to_datetime(
    cloud_api_df['timestamp'].astype(str).str.replace('Z', '', regex=False),
    utc=True,
    errors='coerce'
)
cloud_api_df['hour'] = cloud_api_df['timestamp'].dt.hour

## 3. Investigating IAM events

**Goal:** Find suspicious IAM changes.

### First Hint
Look for events that change who can do what: new keys, new roles, new policies, MFA changes.

### Second Hint
Filter `cloud_iam_df` by `event_name` for values like `CreateAccessKey`, `AttachRolePolicy`, `UpdateAssumeRolePolicy`, `DeactivateMFADevice`.

### Reveal (optional)

In [ ]:
# Example IAM filtering pattern
susp_iam = cloud_iam_df[cloud_iam_df['event_name'].isin([
    'CreateAccessKey', 'AttachRolePolicy', 'UpdateAssumeRolePolicy', 'DeactivateMFADevice'
])]
susp_iam.head()

## 4. Investigating CloudTrail API calls

**Goal:** Find suspicious API usage patterns.

### First Hint
Look for enumeration (List*), discovery (Describe*), and data access (GetObject, GetParameter, etc.).

### Second Hint
Filter `cloud_api_df` by `event_name` and consider unusual regions or principals.

### Reveal (optional)

In [ ]:
# Example API filtering pattern
susp_api = cloud_api_df[cloud_api_df['event_name'].str.startswith('List') |
                        cloud_api_df['event_name'].str.startswith('Describe') |
                        cloud_api_df['event_name'].isin(['GetObject', 'GetParameter'])]
susp_api.head()

## 5. Correlating IAM and API events

**Goal:** Connect configuration changes to subsequent activity.

### First Hint
Ask: did a user or role get new privileges, and then start making unusual API calls?

### Second Hint
Try grouping by `user` or `role` and comparing timestamps between IAM changes and API calls.

### Reveal (optional)

In [ ]:
# Example correlation idea (conceptual, not a full solution)
# 1. Identify users/roles in suspicious IAM events
actors = susp_iam['principal'].unique().tolist() if 'principal' in susp_iam.columns else []

# 2. Filter API events by those actors
if actors:
    related_api = cloud_api_df[cloud_api_df['principal'].isin(actors)]
    related_api.head()
else:
    print("Adjust column names (e.g., 'user', 'role', 'principal') based on the dataset.")

## 6. Extracting IOCs

**Goal:** Turn your findings into concrete artifacts.

### First Hint
Anything you would put into a detection rule, alert, or block list is a good IOC candidate.

### Second Hint
Look at suspicious users, roles, ARNs, IPs (if present), and possibly regions.

### Reveal (optional)

In [ ]:
# Example IOC extraction pattern
susp_users = susp_iam['user'].unique().tolist() if 'user' in susp_iam.columns else []
susp_roles = susp_iam['role'].unique().tolist() if 'role' in susp_iam.columns else []
susp_users, susp_roles

## 7. MITRE ATT&CK mapping

**Goal:** Describe the behavior using ATT&CK techniques.

### First Hint
Think about: valid account use, account manipulation, discovery, and data access.

### Second Hint
Common cloud‑relevant techniques:
- `T1078` (Valid Accounts)
- `T1098` (Account Manipulation)
- `T1087` (Account Discovery)
- `T1530` (Data from Cloud Storage Object)

### Reveal (optional)

In [ ]:
# Example mapping (you should choose based on your findings)
example_mitre = ["T1078", "T1098", "T1530"]
example_mitre

## 8. Writing the triage summary

**Goal:** Tell the story of the incident.

### First Hint
Explain what happened, how you know, and why it matters.

### Second Hint
Include:
- Initial access (if visible)
- Privilege escalation or misuse
- Data access or exfiltration
- Key IOCs and MITRE techniques

### Reveal (optional)

In [ ]:
# Example triage summary structure
example_summary = """
An IAM user appears to have been compromised and used to modify account
permissions, create new access keys, and assume a high‑privilege role.
Shortly after, the actor performed broad enumeration of IAM and S3 resources
and accessed sensitive objects, suggesting data exfiltration risk.
"""
example_summary